# Load PEFT Model

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

train_model_output_dir = "llama-7-int4-ner-a/checkpoint-119"

train_model = AutoPeftModelForCausalLM.from_pretrained(
    train_model_output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    device_map="auto"
)
train_tokenizer = AutoTokenizer.from_pretrained(train_model_output_dir)

In [2]:
from datasets import load_dataset
test_dataset = load_dataset('json', data_files='data/a/test.jsonl',split="train")

In [3]:
len(test_dataset)

16454

# Save the results to csv

In [ ]:
from transformers import (
    pipeline,
)
from tqdm import tqdm
import csv 

results = []

with open('results/systema_1epoch.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter='|')
    writer.writerow(['sentence', 'gold', 'predicted'])
        
    for item in tqdm(test_dataset):
        q = item['instruction']
        gold = item['output']

        pipe = pipeline(task="text-generation", model=train_model, tokenizer=train_tokenizer, max_length=200)
        result = pipe(f"<s>[INST] {q }[/INST]")
        try:
            pred = result[0]['generated_text'].split('\\n')[1].strip()
        
            writer.writerow([q,gold,pred])
            csvfile.flush()
            results.append([q, gold, pred])
        except:
            pass